# Check for negative values

In [2]:
# load_data: returns demand_kBtu data frame with indicated data specifications
def load_data(tags, chw, steam, elec, start = 'NA', end = 'NA'):
    
    
    # load predefined data frames containing year information on which to train the model
    if (start == 'NA') | (end == 'NA'):
        if elec:
            elec_tags = pd.read_csv('elec_tags.csv')
            start = elec_tags[elec_tags['Electricity_Demand_kBtu'] == tags[0]]['start'].tolist()[0]
            end = elec_tags[elec_tags['Electricity_Demand_kBtu'] == tags[0]]['end'].tolist()[0]

        if chw:
            chw_tags = pd.read_csv('chw_tags.csv')
            start = chw_tags[chw_tags['ChilledWater_Demand_kBtu'] == tags[0]]['start'].tolist()[0]
            end = chw_tags[chw_tags['ChilledWater_Demand_kBtu'] == tags[0]]['end'].tolist()[0]

        if steam:
            steam_tags = pd.read_csv('steam_tags.csv')
            start = steam_tags[steam_tags['Steam_Demand_kBtu'] == tags[0]]['start'].tolist()[0]
            end = steam_tags[steam_tags['Steam_Demand_kBtu'] == tags[0]]['end'].tolist()[0]
            
    # load year data from pi client        
    calc = 'summary' 
    interval = '1h'  
    chunk_size = 10
    weight = 'TimeWeighted'
    summary_calc = 'average' 
    max_count = round(1500000/len(tags))

    df = pi.get_stream_by_point(tags, start=start, end=end, _convert_cols=False, calculation=calc, 
                                interval=interval, _weight=weight, _summary_type=summary_calc, _max_count=max_count, 
                                _chunk_size=chunk_size)
    return df



In [3]:
# prep_df: collects and transforms data as needed for each utility to build predictive model. Returns data frame ready to be cleaned
def prep_df(df, chw, steam, elec, train = True):
    if train:    
        # determine start and end date for training data:
        start = df.index[0]
        end = df.index[-1]
    else:
        start = '*-1mo'
        end = '*'
    
    # add outside temperature regardless of the utility 
    tag = pi.search_by_point(['aiTIT4045'])
    calc = 'summary' 
    interval = '1h'   
    chunk_size = 10
    weight = 'TimeWeighted'
    summary_calc = 'average' 
    max_count = round(1500000/len(tag))

    temp = pi.get_stream_by_point(tag, start=start, end=end, _convert_cols=False, calculation=calc, 
                                interval=interval, _weight=weight, _summary_type=summary_calc, _max_count=max_count, 
                                _chunk_size=chunk_size)

    df['aiTIT4045'] = temp.aiTIT4045
    
    # add holidays variable regardless of utility
    #tag = pi.search_by_point(['ACE_Holidays_Boolean'])
    #calc = 'summary' 
    #interval = '1h'   
    #chunk_size = 10
    #weight = 'TimeWeighted'
    #summary_calc = 'average' 
    #max_count = round(1500000/len(tag))

    #hol = pi.get_stream_by_point(tag, start=start, end=end, _convert_cols=False, calculation=calc, 
    #                            interval=interval, _weight=weight, _summary_type=summary_calc, _max_count=max_count, 
    #                            _chunk_size=chunk_size)

    #df['Holiday'] = hol.ACE_Holidays_Boolean
    
    #add hour regardless of utility
    df.reset_index(inplace=True)
    df['Hour'] = pd.to_datetime(df['Timestamp']).apply(lambda x: '{hour}'.format(hour = x.hour))
    df['Hour'] = df['Hour'].astype('int64') 
    
    # add Heating Degree Hour, Cooling Degree Hour suqared and month if this is a chilled water model
    if chw:
        #add HDH
        df['HDH'] = 65 - df['aiTIT4045']
        df.loc[df['HDH'] < 0, 'HDH'] = 0

        #add CDH
        df['CDH'] = (df['aiTIT4045'] - 65)**2
        df.loc[df['CDH'] < 0, 'CDH'] = 0
        
        #add month
        df['Month'] = pd.to_datetime(df['Timestamp']).apply(lambda x: '{month}'.format(month = x.month))
        df['Month'] = df['Month'].astype('int64')
    
    # add Heating Degree Hour squared, Cooling Degree Hour, weekend indicator variable and month if this is a steam model
    if steam:
        # add HDH
        df['HDH'] = (65 - df['aiTIT4045'])**2
        df.loc[df['HDH'] < 0, 'HDH'] = 0
        
        # add CDH
        df['CDH'] = df['aiTIT4045'] - 65
        df.loc[df['CDH'] < 0, 'CDH'] = 0
        
        # add weekend/weekday
        df['DayOfWeek'] = df['Timestamp'].dt.strftime('%w')
        df['DayOfWeek'] = df['DayOfWeek'].astype('int64')
        df['Weekend'] = 0
        df.loc[(df['DayOfWeek'] == 0) | (df['DayOfWeek'] == 6), 'Weekend'] = 1
        df.loc[(df['DayOfWeek'] == 1) | (df['DayOfWeek'] == 2) | (df['DayOfWeek'] == 3) | (df['DayOfWeek'] == 4) | (df['DayOfWeek'] == 5), 'Weekend'] = 0
        df.drop(['DayOfWeek'], inplace = True, axis = 1)
                                       
        # add month
        df['Month'] = pd.to_datetime(df['Timestamp']).apply(lambda x: '{month}'.format(month = x.month)) 
        df['Month'] = df['Month'].astype('int64')                                
           
    # add Heating Degree Hour, Cooling Degree Hour and if available WiFi occupancy if this is an electricity model
    if elec:
        #add HDH
        df['HDH'] = 65 - df['aiTIT4045']
        df.loc[df['HDH'] < 0, 'HDH'] = 0
        
        #add CDH
        df['CDH'] = df['aiTIT4045'] - 65
        df.loc[df['CDH'] < 0, 'CDH'] = 0
        
        # load predefined data frame to find if building has WiFi occupancy data
        elec_tags = pd.read_csv('elec_tags.csv')
        row = elec_tags.loc[elec_tags['Electricity_Demand_kBtu'] == elec[0]]
        status = row.Status.tolist()
        
        if status[0] == 'complete':
            # add WiFi
            # load predefined data frame containing matched WiFi tags since names are inconsistent
            wifi_matched_tags = pd.read_csv('wifi_matched_tags.csv')

            wifi_tag = wifi_matched_tags[wifi_matched_tags['Electricity_Tag'] == elec[0]]['Wi-Fi Tags'].to_list()

            df.set_index('Timestamp',inplace=True)
            start = df.index[0]
            end = df.index[-1]
            calc = 'summary' 
            interval = '1h'   
            chunk_size = 10
            weight = 'TimeWeighted'
            summary_calc = 'average' 
            max_count = round(1500000/len(wifi_tag))

            wifi = pi.get_stream_by_point(wifi_tag, start=start, end=end, _convert_cols=False, calculation=calc, 
                                        interval=interval, _weight=weight, _summary_type=summary_calc, _max_count=max_count, 
                                        _chunk_size=chunk_size)

            wifi.reset_index(inplace = True)
            df.reset_index(inplace = True)
            #df = df.iloc[:-1 , :]
            
            # Certain buildings contain multiple WiFi occupancy locations. Add those that do
            if len(wifi_tag) > 1:
                col = np.zeros(len(wifi))
                for i in wifi_tag:
                    col = wifi[i] + col
                df['Wifi_Occupancy_Count'] = col
            else:
                df['Wifi_Occupancy_Count'] = wifi[[wifi_tag[0]]]
                
            
            #df.reset_index(inplace = True)
            
        
    return df                              
                                       
        

In [4]:
# clean_data: performs data cleaning according to the needs of each utility
def clean_data(df, chw, steam, elec, train = True):
    df = df[(df['aiTIT4045'] > 0) & (df['aiTIT4045'] < 120)]
    df.dropna(axis=0, inplace=True)
    df.set_index('Timestamp',inplace = True)
    
    
    if train:
        demand_tag = [string for string in df.columns if string.endswith("_Demand_kBtu")]
        # generaly remove negative and zero values as well as temperatures that do not make sense
        df = df[df[demand_tag[0]] > 0]
        
        if chw:
            df_out = df[[demand_tag[0], 'aiTIT4045']].copy()
        
        if steam: 
            # remove spikes in steam that go over 6500
            df = df[df[steam[0]] < 6500]
            df_out = df[[demand_tag[0], 'aiTIT4045']].copy()
            
        if elec:
            elec_tags = pd.read_csv('elec_tags.csv')
            row = elec_tags.loc[elec_tags['Electricity_Demand_kBtu'] == elec[0]]
            status = row.Status.tolist()
            if status[0] == 'complete':
                wifi = [string for string in df.columns if string.endswith("_Count")]
                df_out = df[[demand_tag[0], 'aiTIT4045', wifi[0]]].copy()
            else:
                df_out = df[[demand_tag[0], 'aiTIT4045']].copy()
        
    else:
        df_out = df[['aiTIT4045']].copy()
    
    # remove outliers 
    IQR = df_out.quantile(0.75) - df_out.quantile(0.25)
    df_final = df_out[~((df_out < (df_out.quantile(0.25) - 3 * IQR)) | (df_out > (df_out.quantile(0.75) + 3 * IQR)))]

    df_final.dropna(axis=0, inplace=True)

    df_final.reset_index(inplace=True)
    df = df.loc[df_final['Timestamp']]
    df.reset_index(inplace = True)

    df.drop(['aiTIT4045'], inplace=True, axis=1)
        
    return df 
      

In [1]:
# split_data: use sklearn train_test_split to split data 
#def split_data(df):
#    from sklearn.model_selection import train_test_split 
#    demand = [string for string in df.columns if string.endswith("_Demand_kBtu")]
#    
#    x = df.drop(demand, axis = 1)
#    y = df[demand].values.reshape(-1,1)
#
#    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.0, shuffle = False)
#    
#    # mantain dates for later use
#    dates_train = X_train['Timestamp']
#    dates_test = X_test['Timestamp']
#    dates_val = X_val['Timestamp']
#    X_train.drop('Timestamp', axis = 1, inplace = True)
#    X_test.drop('Timestamp', axis = 1, inplace = True)
#    X_val.drop('Timestamp', axis = 1, inplace = True)
#    
#    return X_train, X_test, X_val, y_train, y_test, y_val, dates_train, dates_test, dates_val    
    

In [6]:
# get_predictions: builds and fits an optimized model on one year of data and predicts. Results are measured and there are two cases for the predictions. If test = True it predicts on the last month of the year selected. If test = false it calculates results for training, test and validation sets
def get_predictions(X_train = 'NA', y_train = 'NA', last_month_x = 'NA'):
    from sklearn.ensemble import GradientBoostingRegressor
    from sklearn.model_selection import RandomizedSearchCV
    
    # parameters to use in tuning
    param_grid = {'criterion':['friedman_mse', 'mse'], 'loss':['ls','lad'],'learning_rate':[0.1,0.01, 0.001, 0.05],                               
                  'n_estimators':np.linspace(10, 200).astype(int), 'random_state':[1], 'min_samples_split': [2, 5, 10],
                 'max_depth': [None] + list(np.linspace(3, 20).astype(int)), 'max_leaf_nodes': [None] + list(np.linspace(10, 50, 500).astype(int))}

    estimator = GradientBoostingRegressor()
    
    # randomized search to optimized model
    rs = RandomizedSearchCV(estimator, param_grid, n_jobs = -1, 
                            scoring = 'neg_root_mean_squared_error', cv = 5, 
                            n_iter = 10, verbose = 1)
    rs.fit(X_train, y_train)
    
    # save best parameters
    parameters = rs.best_params_
    
    # build and fit optimized model
    model = GradientBoostingRegressor(**parameters)
    model.fit(X_train, y_train)
    
    #predict on last month of data
    pred_month = model.predict(last_month_x)
        
    # remove negative predictions since it's not meaningful. Added to improve the results of those steam and chilled water models that did not pass the threshhold 
    pred_month1 = list()
    for i in range(len(pred_month)):
        if pred_month[i] > 0:
            #is_.append(i)
            pred_month1.append(pred_month[i])
        else:
            pred_month1.append(0)
                
    return pred_month1
        
   

In [53]:
# demandkBtu_model: puts together previous functions to implement the process from beginning to end. returns visualizations and results depending of the value for 'test'
def demandkBtu_model(tags, start = 'NA', end = 'NA'):
    import pandas as pd
    # loads predefined data frames containing tag information such as year of optimal data and quality of model/data
    elec_tags = pd.read_csv('elec_tags.csv')
    chw_tags = pd.read_csv('chw_tags.csv')
    steam_tags = pd.read_csv('steam_tags.csv')
    elec_tags = elec_tags[(elec_tags['Status'] == 'complete') | (elec_tags['Status'] == 'incomplete')]
    chw_tags = chw_tags[chw_tags['Status'] == 'complete']
    steam_tags = steam_tags[steam_tags['Status'] == 'complete']
    possible_tags = elec_tags['Electricity_Demand_kBtu'].tolist() + chw_tags['ChilledWater_Demand_kBtu'].tolist() + steam_tags['Steam_Demand_kBtu'].tolist()
    
    chw = [string for string in tags if string.endswith("ChilledWater_Demand_kBtu")]
    steam = [string for string in tags if string.endswith("Steam_Demand_kBtu")]
    elec = [string for string in tags if string.endswith("Electricity_Demand_kBtu")]

    
    # obtain the data frame to train the  model
    df_train = clean_data(prep_df(load_data(tags, chw = chw, steam = steam, elec = elec), train = True, chw = chw, steam = steam, elec = elec), chw = chw, steam = steam, elec = elec)
    
    # obtain data frame with last month's data to predict demand_kBtu
    last_month_x = pd.DataFrame()
    last_month_x = prep_df(last_month_x, train = False, chw = chw, steam = steam, elec = elec)
    dates_month = last_month_x['Timestamp']
    dates_month.reset_index(inplace = True, drop = True)
    last_month_x = clean_data(last_month_x, train = False, chw = chw, steam = steam, elec = elec)


    # predict the last month of observations
    dates_month2 = last_month_x['Timestamp']
    dates_month2.reset_index(inplace = True, drop = True)
    
    # split x and y training data
    from sklearn.model_selection import train_test_split 
    
    x = df_train.drop(tags[0], axis = 1)
    y = df_train[tags[0]].values.reshape(-1,1)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.0001, shuffle = False)
    
    y_dummy = dates_month2.values.reshape(-1,1)
    last_month_x, dummyyy, dummyy, dummy = train_test_split(last_month_x, y_dummy, test_size = 0.001, shuffle = False)
    
    dates_month2 = last_month_x['Timestamp']
    dates_month2.reset_index(inplace = True, drop = True)
    last_month_x.drop('Timestamp', axis = 1, inplace = True)
    
    X_train.drop('Timestamp', axis = 1, inplace = True)
        
    # build model using data for training and last_month_x for predictions
    pred_month = get_predictions(X_train = X_train, y_train = y_train, last_month_x = last_month_x)
                            
    # returns predictions, dates of said predictions
    return pred_month, dates_month
    
    

In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ecotools.pi_client import pi_client

sns.set('paper')
# Option Paramater time_zone
# default timezone is time_zone='America/Los_Angeles' use https://www.iana.org/time-zones 

#pi = pi_client(username= r'ad3\username')
pi = pi_client(username = r'ad3\dlrueda')

ad3\dlrueda, please provide your password (text will be hidden) 
>>········
save password? [y/n] /n after saving, set username = 'file' to use this feature 
>>n


In [34]:
tags = ['AcademicSurge_ChilledWater_Demand_kBtu']
predictions2, dates2 = demandkBtu_model(tags) 

18:11:17: Sending API request... Chunk 1 of 1
18:11:18: Response recieved for 1 tags! (0.63)
18:11:18: Sending API request... Chunk 1 of 1
18:11:20: Response recieved for 1 tags! (1.84)


<ipython-input-7-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


18:11:20: Sending API request... Chunk 1 of 1
18:11:20: Response recieved for 1 tags! (0.16)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [35]:
len(predictions2)

719

In [36]:
len(dates2)

719

# Try it in all

In [8]:
elec_tags = pd.read_csv('elec_tags.csv')
chw_tags = pd.read_csv('chw_tags.csv')
steam_tags = pd.read_csv('steam_tags.csv')
elec_tags = elec_tags[(elec_tags['Status'] == 'complete') | (elec_tags['Status'] == 'incomplete')]
chw_tags = chw_tags[chw_tags['Status'] == 'complete']
steam_tags = steam_tags[steam_tags['Status'] == 'complete']
possible_tags = elec_tags['Electricity_Demand_kBtu'].tolist() + chw_tags['ChilledWater_Demand_kBtu'].tolist() + steam_tags['Steam_Demand_kBtu'].tolist()
    

In [38]:
len(possible_tags)

188

In [39]:
len(chw_tags)

49

In [40]:
len(steam_tags)

16

In [41]:
len(elec_tags)

123

In [46]:
########################

#dataframe for predictions
predictions_steam = pd.DataFrame()

#dataframe for dates
dates_steam = pd.DataFrame()

# for status == complete or incomplete
for i in steam_tags['Steam_Demand_kBtu'].tolist():
    tag = list()
    tag.append(i)
    predictions_steam[i], dates_steam[i] = demandkBtu_model(tag) 
 

18:32:09: Sending API request... Chunk 1 of 1
18:32:10: Response recieved for 1 tags! (1.2)
18:32:10: Sending API request... Chunk 1 of 1
18:32:11: Response recieved for 1 tags! (0.58)


<ipython-input-7-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


18:32:11: Sending API request... Chunk 1 of 1
18:32:11: Response recieved for 1 tags! (0.19)
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


18:32:42: Sending API request... Chunk 1 of 1
18:32:43: Response recieved for 1 tags! (0.96)
18:32:44: Sending API request... Chunk 1 of 1
18:32:47: Response recieved for 1 tags! (3.18)


...

<ipython-input-7-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


18:38:52: Sending API request... Chunk 1 of 1
18:38:52: Response recieved for 1 tags! (0.16)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [47]:
########################

#dataframe for predictions
predictions_chw = pd.DataFrame()

#dataframe for dates
dates_chw = pd.DataFrame()

# for status == complete or incomplete
for i in chw_tags['ChilledWater_Demand_kBtu'].tolist():
    tag = list()
    tag.append(i)
    predictions_chw[i], dates_chw[i] = demandkBtu_model(tag) 
 

18:40:49: Sending API request... Chunk 1 of 1
18:40:50: Response recieved for 1 tags! (0.97)
18:40:50: Sending API request... Chunk 1 of 1
18:40:52: Response recieved for 1 tags! (1.63)


<ipython-input-7-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


18:40:52: Sending API request... Chunk 1 of 1
18:40:52: Response recieved for 1 tags! (0.16)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

<ipython-input-7-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


19:00:27: Sending API request... Chunk 1 of 1
19:00:27: Response recieved for 1 tags! (0.16)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [68]:
elec_tags = pd.read_csv('elec_tags.csv')
run_tags = elec_tags.loc[(elec_tags['Status'] == 'complete') | (elec_tags['Status'] == 'incomplete'), 'Electricity_Demand_kBtu'].tolist()


In [40]:
run_tags = run_tags[0:9]

In [41]:
run_tags

['AcademicSurge_Electricity_Demand_kBtu',
 'ARC_Electricity_Demand_kBtu',
 'ArtAnnex_Electricity_Demand_kBtu',
 'Asmundson_Electricity_Demand_kBtu',
 'Bainer_Electricity_Demand_kBtu',
 'California_Electricity_Demand_kBtu',
 'CampusDataCenter_Electricity_Demand_kBtu',
 'Chemistry_Electricity_Demand_kBtu',
 'Chemistry.Annex_Electricity_Demand_kBtu']

In [69]:
########################

#dataframe for predictions
predictions_elec = pd.DataFrame(index = range(720))

#dataframe for dates
dates_elec = pd.DataFrame()

# for status == complete or incomplete
for i in run_tags:
    tag = list()
    tag.append(i)
    dates = pd.DataFrame()
    predictions = pd.DataFrame()
    predictions[i], dates[i] = demandkBtu_model(tag) 
    #predictions['Timestamp'] = dates[[i]]
    #predictions_elec.set_index()
    predictions_elec[i] = predictions[[i]]
    #predictions_elec['Timestamp'] = dates[[i]]
    #predictions_elec.set_index('Timestamp', inplace = True)
    #predictions.set_index('Timestamp', inplace = True)
    #predictions_elec = pd.concat([predictions_elec, predictions], axis = 1)
    #predictions_elec.merge(predictions, left_index = True, right_index = True)
    
predictions_elec['Timestamp'] = dates[[i]]
predictions_elec.set_index('Timestamp', inplace = True)

16:20:18: Sending API request... Chunk 1 of 1
16:20:19: Response recieved for 1 tags! (1.17)
16:20:19: Sending API request... Chunk 1 of 1
16:20:21: Response recieved for 1 tags! (1.6)
16:20:21: Sending API request... Chunk 1 of 1
16:20:22: Response recieved for 1 tags! (0.93)
16:20:22: Sending API request... Chunk 1 of 1


<ipython-input-4-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


16:20:22: Response recieved for 1 tags! (0.16)
16:20:22: Sending API request... Chunk 1 of 1
16:20:22: Response recieved for 1 tags! (0.19)
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


16:20:52: Sending API request... Chunk 1 of 1
16:20:53: Response recieved for 1 tags! (1.44)
16:20:53: Sending API request... Chunk 1 of 1
16:20:54: Response recieved for 1 tags! (0.7)


...

<ipython-input-4-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


17:12:42: Sending API request... Chunk 1 of 1
17:12:42: Response recieved for 1 tags! (0.17)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [70]:
predictions_elec


,AcademicSurge_Electricity_Demand_kBtu,ARC_Electricity_Demand_kBtu,ArtAnnex_Electricity_Demand_kBtu,Asmundson_Electricity_Demand_kBtu,Bainer_Electricity_Demand_kBtu,California_Electricity_Demand_kBtu,CampusDataCenter_Electricity_Demand_kBtu,Chemistry_Electricity_Demand_kBtu,Chemistry.Annex_Electricity_Demand_kBtu,Cole.B_Electricity_Demand_kBtu,...,Veihmeyer_Hall_Electricity_Demand_kBtu,VM.EAPL_Electricity_Demand_kBtu,VM3A_Electricity_Demand_kBtu,VM3B_Electricity_Demand_kBtu,VMSSAC_Electricity_Demand_kBtu,VMTHSurgical_Electricity_Demand_kBtu,VMTHWard_Electricity_Demand_kBtu,VMTH_Electricity_Demand_kBtu,WHNRC_Electricity_Demand_kBtu,WSEH_Electricity_Demand_kBtu
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-11-29 17:12:42.584470,998.434665,831.368515,80.567865,209.109746,1361.776048,60.526082,956.037799,1628.235601,1257.646201,116.751259,...,144.474559,50.240067,1681.718262,1158.359915,71.482184,82.491362,82.269602,1112.151449,471.119272,110.564124
2021-11-29 18:12:42.584470,970.206955,799.773633,80.430128,201.584620,1249.503995,58.867558,956.375303,1604.147086,1206.879473,116.856660,...,138.772871,50.622048,1579.410104,1028.269029,68.807410,82.717294,80.495906,1081.628741,451.822711,110.564124
2021-11-29 19:12:42.584470,944.750736,773.518770,65.782310,188.956286,1107.789486,58.970404,956.375303,1451.570985,1133.094898,113.398549,...,133.837628,50.622048,1524.596213,995.407643,68.046380,83.111545,84.070381,1072.611522,439.027137,110.106557
2021-11-29 20:12:42.584470,908.809670,751.402181,70.564531,187.286898,1053.736557,56.023979,956.344106,1413.599836,1053.454524,111.004708,...,131.537748,50.622048,1477.537858,952.430427,63.847476,83.067109,82.864134,1065.449848,429.931272,110.106557
2021-11-29 21:12:42.584470,884.778354,753.793378,63.985656,180.477267,1022.934774,50.585049,956.277007,1389.850820,1033.690316,107.668395,...,129.395412,50.361476,1453.951004,935.274492,61.951082,82.044355,83.187942,1051.846357,428.005681,110.106557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-29 12:12:42.584470,863.717136,876.369958,96.958224,190.904718,1092.882094,38.777989,956.021920,1438.683161,1036.460363,123.110429,...,141.525726,50.313336,1733.940214,1196.112548,89.722949,85.442645,96.464261,1171.501245,494.775139,115.579020
2021-12-29 13:12:42.584470,845.121409,876.080809,65.857668,195.333611,1048.404019,38.777989,956.021920,1436.468536,1050.462023,122.997680,...,141.525726,49.770845,1733.940214,1196.112548,89.722949,87.747119,92.864162,1169.589698,497.095054,115.579020
2021-12-29 14:12:42.584470,NaN,780.016097,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,141.525726,49.770845,1767.291888,1194.824746,104.903279,83.281794,83.049956,1155.950089,490.930477,113.412110


In [66]:
predictions_elec['Timestamp'] = dates[[i]]
predictions_elec.set_index('Timestamp', inplace = True)

In [65]:
dates

,Chemistry.Annex_Electricity_Demand_kBtu
0,2021-11-29 16:12:05.224245100
1,2021-11-29 17:12:05.224245100
2,2021-11-29 18:12:05.224245100
3,2021-11-29 19:12:05.224245100
4,2021-11-29 20:12:05.224245100
...,...
715,2021-12-29 11:12:05.224245100
716,2021-12-29 12:12:05.224245100
717,2021-12-29 13:12:05.224245100
718,2021-12-29 14:12:05.224245100


In [59]:
dates

,AcademicSurge_Electricity_Demand_kBtu
0,2021-11-29 15:49:36.763168900
1,2021-11-29 16:49:36.763168900
2,2021-11-29 17:49:36.763168900
3,2021-11-29 18:49:36.763168900
4,2021-11-29 19:49:36.763168900
...,...
712,2021-12-29 08:49:36.763168900
713,2021-12-29 09:49:36.763168900
714,2021-12-29 10:49:36.763168900
715,2021-12-29 11:49:36.763168900


In [36]:
pd.concat([predictions, predictions2], axis = 1)

,WSEH_Electricity_Demand_kBtu,WSEH_Electricity_Demand_kBtu
Timestamp,,
2021-11-29 14:34:05.614123,131.279622,131.279622
2021-11-29 15:34:05.614123,125.826988,125.826988
2021-11-29 16:34:05.614123,114.078531,114.078531
2021-11-29 17:34:05.614123,112.787795,112.787795
2021-11-29 18:34:05.614123,111.331685,111.331685
...,...,...
2021-12-29 08:34:05.614123,113.700840,113.700840
2021-12-29 09:34:05.614123,117.147988,117.147988
2021-12-29 10:34:05.614123,117.147988,117.147988


# ---------------------------------
# Run all

In [71]:
elec_tags = pd.read_csv('elec_tags.csv')
chw_tags = pd.read_csv('chw_tags.csv')
steam_tags = pd.read_csv('steam_tags.csv')
elec_tags = elec_tags[(elec_tags['Status'] == 'complete') | (elec_tags['Status'] == 'incomplete')]
chw_tags = chw_tags[chw_tags['Status'] == 'complete']
steam_tags = steam_tags[steam_tags['Status'] == 'complete']
possible_tags = elec_tags['Electricity_Demand_kBtu'].tolist() + chw_tags['ChilledWater_Demand_kBtu'].tolist() + steam_tags['Steam_Demand_kBtu'].tolist()
    

In [75]:
########################

#dataframe for predictions
predictions_all = pd.DataFrame(index = range(718))

#dataframe for dates
#dates_elec = pd.DataFrame()

# for status == complete or incomplete
for i in possible_tags:
    tag = list()
    tag.append(i)
    dates = pd.DataFrame()
    predictions = pd.DataFrame()
    predictions[i], dates[i] = demandkBtu_model(tag) 
    predictions_all[i] = predictions[[i]]
    
predictions_all['Timestamp'] = dates[[i]]
predictions_all.set_index('Timestamp', inplace = True)

17:27:31: Sending API request... Chunk 1 of 1
17:27:32: Response recieved for 1 tags! (0.6)
17:27:32: Sending API request... Chunk 1 of 1
17:27:34: Response recieved for 1 tags! (1.66)
17:27:34: Sending API request... Chunk 1 of 1
17:27:34: Response recieved for 1 tags! (0.62)
17:27:35: Sending API request... Chunk 1 of 1


<ipython-input-4-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


17:27:35: Response recieved for 1 tags! (0.16)
17:27:35: Sending API request... Chunk 1 of 1
17:27:35: Response recieved for 1 tags! (0.17)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


...

<ipython-input-4-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


18:44:57: Sending API request... Chunk 1 of 1
18:44:57: Response recieved for 1 tags! (0.16)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [73]:
len(possible_tags)

188

In [76]:
predictions_all

,AcademicSurge_Electricity_Demand_kBtu,ARC_Electricity_Demand_kBtu,ArtAnnex_Electricity_Demand_kBtu,Asmundson_Electricity_Demand_kBtu,Bainer_Electricity_Demand_kBtu,California_Electricity_Demand_kBtu,CampusDataCenter_Electricity_Demand_kBtu,Chemistry_Electricity_Demand_kBtu,Chemistry.Annex_Electricity_Demand_kBtu,Cole.B_Electricity_Demand_kBtu,...,Robbins_Steam_Demand_kBtu,SciLab_Steam_Demand_kBtu,SegundoServices_Steam_Demand_kBtu,Tercero3_Steam_Demand_kBtu,TheGrove_Steam_Demand_kBtu,Thurman_Steam_Demand_kBtu,Tupper_Steam_Demand_kBtu,VM3A_Steam_Demand_kBtu,WHNRC_Steam_Demand_kBtu,Wickson_Steam_Demand_kBtu
Timestamp,,,,,,,,,,,,,,,,,,,,,
2021-11-29 18:44:57.242940600,949.342014,798.127189,78.787297,201.372949,1227.605252,56.385651,956.543923,1598.988104,1204.380732,116.449461,...,1012.044546,1504.854840,1320.794350,955.475079,1570.231075,502.014785,3117.576882,1284.209787,495.850772,1095.437805
2021-11-29 19:44:57.242940600,945.194386,776.375845,66.478132,188.101888,1104.576922,56.345846,956.543923,1431.860909,1123.236361,113.307656,...,1122.021432,1754.996710,1368.363941,1020.033048,1572.470594,603.352243,3438.421698,1475.541528,488.246282,1172.648665
2021-11-29 20:44:57.242940600,903.687602,752.436515,70.884400,186.207465,1045.903613,54.260512,956.543923,1398.098491,1058.072426,110.267025,...,1129.621061,1881.963786,1543.858326,1225.632460,1652.081429,685.526545,3654.861810,1591.263210,494.760735,1245.871290
2021-11-29 21:44:57.242940600,875.685505,751.345678,65.580647,180.933939,1034.020338,48.891276,955.746084,1396.903244,1041.175467,107.860998,...,1171.588735,2011.288004,1683.484035,1329.439234,1636.179952,712.351840,3789.748600,1756.685152,506.154955,1247.644316
2021-11-29 22:44:57.242940600,874.623254,737.989657,64.116525,180.786189,1011.690941,45.582468,955.746084,1388.278949,1060.711142,104.517337,...,1229.349318,2268.616905,1749.385812,1337.081152,1695.860629,719.675773,3905.585193,1797.849711,532.112822,1272.658310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-29 13:44:57.242940600,862.799187,776.917898,73.408524,191.301679,1033.813892,38.955338,955.746084,1443.826234,1047.313320,123.000922,...,1791.219347,3971.521220,2614.891543,1560.938866,2198.643695,1188.747631,5951.467970,3418.256979,994.593365,2286.480187
2021-12-29 14:44:57.242940600,NaN,890.750185,NaN,NaN,NaN,NaN,955.746084,1452.819177,1045.091239,120.808539,...,1790.099379,3940.805794,2517.583829,1509.109228,2198.643695,1149.991154,5670.502454,3455.191264,974.826688,2308.287035
2021-12-29 15:44:57.242940600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1794.537421,3940.805794,2501.360058,1510.344817,2198.643695,1149.545094,5666.078116,3437.870677,975.967846,2317.811760


# ---------------------------------------
# Prediction data

In [45]:
tags = ['AcademicSurge_ChilledWater_Demand_kBtu']
   
chw = [string for string in tags if string.endswith("ChilledWater_Demand_kBtu")]
steam = [string for string in tags if string.endswith("Steam_Demand_kBtu")]
elec = [string for string in tags if string.endswith("Electricity_Demand_kBtu")]



In [47]:
last_month_x = pd.DataFrame()
last_month_x = clean_data(prep_df(last_month_x, train = False, chw = chw, steam = steam, elec = elec), train = False, chw = chw, steam = steam, elec = elec)
last_month_x


00:36:44: Sending API request... Chunk 1 of 1
00:36:44: Response recieved for 1 tags! (0.16)
00:36:45: Sending API request... Chunk 1 of 1
00:36:45: Response recieved for 1 tags! (0.16)


,Timestamp,Holiday,Hour,HDH,CDH,Month


In [49]:
last_month_x = pd.DataFrame()
prep_df(last_month_x, train = False, chw = chw, steam = steam, elec = elec)

00:38:09: Sending API request... Chunk 1 of 1
00:38:09: Response recieved for 1 tags! (0.16)
00:38:09: Sending API request... Chunk 1 of 1
00:38:09: Response recieved for 1 tags! (0.14)


,Timestamp,aiTIT4045,Holiday,Hour,HDH,CDH,Month
0,2021-11-19 00:38:09.321016600,54.107667,NaN,0,10.892333,118.642926,11
1,2021-11-19 01:38:09.321016600,53.439363,NaN,1,11.560637,133.648334,11
2,2021-11-19 02:38:09.321016600,53.168479,NaN,2,11.831521,139.984897,11
3,2021-11-19 03:38:09.321016600,53.387816,NaN,3,11.612184,134.842814,11
4,2021-11-19 04:38:09.321016600,53.747438,NaN,4,11.252562,126.620154,11
...,...,...,...,...,...,...,...
715,2021-12-18 19:38:09.321016600,41.126203,NaN,19,23.873797,569.958178,12
716,2021-12-18 20:38:09.321016600,40.722953,NaN,20,24.277047,589.374997,12
717,2021-12-18 21:38:09.321016600,40.943534,NaN,21,24.056466,578.713552,12
718,2021-12-18 22:38:09.321016600,41.015818,NaN,22,23.984182,575.241001,12


# ------------------------------------
# Electricity

In [9]:
########################
#elec_tags = pd.read_csv('elec_tags.csv')
#complete = elec_tags.loc[elec_tags['Status'] == 'complete']

#dataframe for predictions
#predictions = pd.DataFrame()

#dataframe for dates
#dates = pd.DataFrame()

#list tags with randomly generated numbers
#from random import randint
tags = ['1850_Research_Park_Electricity_Demand_kBtu']
#for i in range(0):
#num = randint(0, len(elec_tags)-1)
#tags.append(complete['Electricity_Demand_kBtu'][num])

#loop
#for i in tags:
    #tag = list()
    #tag.append(i)
    #status = elec_tags.loc[elec_tags['Electricity_Demand_kBtu'] == i]['Status'].tolist()
    #if status[0] == 'incomplete':
predictions, dates = demandkBtu_model(tags) 
    

13:36:37: Sending API request... Chunk 1 of 1
Encountered an unhandled HTTP error!
(400)
Reason: Bad Request
HTTP response headers: {'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'Content-Security-Policy': "default-src 'self'; object-src 'none'", 'X-Content-Type-Options': 'nosniff', 'cache-control': 'no-cache', 'content-encoding': 'gzip', 'Date': 'Wed, 29 Dec 2021 21:36:36 GMT'}
HTTP response body: {"Errors":["'no data' is not a valid timestamp."]}

piStreamValues is empty!


IndexError: index 0 is out of bounds for axis 0 with size 0

In [39]:
len(predictions)

717

In [40]:
len(dates)

717

In [31]:
dates

0     2021-11-19 18:05:11.124175900
1     2021-11-19 19:05:11.124175900
2     2021-11-19 20:05:11.124175900
3     2021-11-19 21:05:11.124175900
4     2021-11-19 22:05:11.124175900
                   ...             
715   2021-12-19 13:05:11.124175900
716   2021-12-19 14:05:11.124175900
717   2021-12-19 15:05:11.124175900
718   2021-12-19 16:05:11.124175900
719   2021-12-19 17:05:11.124175900
Name: Timestamp, Length: 720, dtype: datetime64[ns]

In [11]:
tags =['1850_Research_Park_Electricity_Demand_kBtu']
   
chw = [string for string in tags if string.endswith("ChilledWater_Demand_kBtu")]
steam = [string for string in tags if string.endswith("Steam_Demand_kBtu")]
elec = [string for string in tags if string.endswith("Electricity_Demand_kBtu")]

#df_train = clean_data(prep_df(load_data(tags, chw = chw, steam = steam, elec = elec), train = True, chw = chw, steam = steam, elec = elec), chw = chw, steam = steam, elec = elec)




In [12]:
df = load_data(tags, chw = chw, steam = steam, elec = elec)
df.head()

13:36:58: Sending API request... Chunk 1 of 1
Encountered an unhandled HTTP error!
(400)
Reason: Bad Request
HTTP response headers: {'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Server': 'Microsoft-HTTPAPI/2.0', 'Content-Security-Policy': "default-src 'self'; object-src 'none'", 'X-Content-Type-Options': 'nosniff', 'cache-control': 'no-cache', 'content-encoding': 'gzip', 'Date': 'Wed, 29 Dec 2021 21:36:58 GMT'}
HTTP response body: {"Errors":["'no data' is not a valid timestamp."]}

piStreamValues is empty!


""


In [37]:
df2 = prep_df(df, train = True, chw = chw, steam = steam, elec = elec)
df2.head()

20:01:05: Sending API request... Chunk 1 of 1
20:01:07: Response recieved for 1 tags! (1.75)
20:01:07: Sending API request... Chunk 1 of 1
20:01:08: Response recieved for 1 tags! (0.57)


,Timestamp,AcademicSurge_Electricity_Demand_kBtu,aiTIT4045,Hour,HDH,CDH,Wifi_Occupancy_Count
0,2018-09-01 00:00:00,793.869591,67.071167,0,0.000000,2.071167,31.194826
1,2018-09-01 01:00:00,793.782155,65.564086,1,0.000000,0.564086,25.623586
2,2018-09-01 02:00:00,793.669571,64.364953,2,0.635047,0.000000,24.000007
3,2018-09-01 03:00:00,800.983966,64.004323,3,0.995677,0.000000,24.166673
4,2018-09-01 04:00:00,796.661268,63.287432,4,1.712568,0.000000,23.499998


In [13]:
wifi = pd.read_csv('wifi_matched_tags.csv')
wifi

,Unnamed: 0,Building Name,Wi-Fi Tags,Electricity_Tag
0,0,Academic Surge Building,AP.ACADMC-SURGE_Total_Count,AcademicSurge_Electricity_Demand_kBtu
1,1,Activities and Recreation Center,AP.ARC_Total_Count,ARC_Electricity_Demand_kBtu
2,2,Art Building,AP.ART_Total_Count,Art_Electricity_Demand_kBtu
3,3,Art Building Annex,AP.ART-ANNEX_Total_Count,ArtAnnex_Electricity_Demand_kBtu
4,4,Asmundson Hall,AP.ASMDSN_Total_Count,Asmundson_Electricity_Demand_kBtu
...,...,...,...,...
70,70,Wellman Hall,AP.WELLMAN-BSMT-NE_Total_Count,Wellman_Kerr_Electricity_Demand_kBtu
71,71,Wellman Hall,AP.WELLMAN-BSMT-SOUTH_Total_Count,Wellman_Kerr_Electricity_Demand_kBtu
72,72,Wickson Hall,AP.WICKSON_Total_Count,Wickson_Electricity_Demand_kBtu
73,73,Wright Hall,AP.WRIGHT-HALL_Total_Count,Wright_Electricity_Demand_kBtu


In [14]:
wifi_tag = wifi[wifi['Electricity_Tag'] == elec[0]]['Wi-Fi Tags'].to_list()
wifi_tag

[]

In [15]:
elec_tags = pd.read_csv('elec_tags.csv')
elec_tags.loc[elec_tags['Electricity_Demand_kBtu'] == elec[0]]


,Electricity_Demand_kBtu,start,end,Status
61,1850_Research_Park_Electricity_Demand_kBtu,no data,no data,no data


In [31]:
start = '2018-09-01'
end = '2019-09-01'
calc = 'summary' 
interval = '1h'   
chunk_size = 10
weight = 'TimeWeighted'
summary_calc = 'average' 
max_count = round(1500000/len(wifi_tag))

wifi_df = pi.get_stream_by_point(wifi_tag, start=start, end=end, _convert_cols=False, calculation=calc, 
                            interval=interval, _weight=weight, _summary_type=summary_calc, _max_count=max_count, 
                            _chunk_size=chunk_size)

19:56:21: Sending API request... Chunk 1 of 1
19:56:21: Response recieved for 1 tags! (0.6)


In [32]:
wifi_df

,AP.ACADMC-SURGE_Total_Count
Timestamp,
2018-09-01 00:00:00,31.194826
2018-09-01 01:00:00,25.623586
2018-09-01 02:00:00,24.000007
2018-09-01 03:00:00,24.166673
2018-09-01 04:00:00,23.499998
...,...
2019-08-31 19:00:00,39.533374
2019-08-31 20:00:00,39.989954
2019-08-31 21:00:00,35.185087


In [20]:
len(wifi_df)

8759

In [21]:
wifi_df.dropna(axis=0, inplace=True)
len(wifi_df)

8759

In [53]:
df3 = clean_data(df2, chw = chw, steam = steam, elec = elec)



<ipython-input-3-91639ad4b722>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(axis=0, inplace=True)


# -------------------------------
# correcting wifi complete incomplete status inaccuracies

In [4]:
wifi_tags = pd.read_csv('wifi_matched_tags.csv')
wifi_tags

,Unnamed: 0,Building Name,Wi-Fi Tags,Electricity_Tag
0,0,Academic Surge Building,AP.ACADMC-SURGE_Total_Count,AcademicSurge_Electricity_Demand_kBtu
1,1,Activities and Recreation Center,AP.ARC_Total_Count,ARC_Electricity_Demand_kBtu
2,2,Art Building,AP.ART_Total_Count,Art_Electricity_Demand_kBtu
3,3,Art Building Annex,AP.ART-ANNEX_Total_Count,ArtAnnex_Electricity_Demand_kBtu
4,4,Asmundson Hall,AP.ASMDSN_Total_Count,Asmundson_Electricity_Demand_kBtu
5,5,Bainer Hall,AP.BAINER_Total_Count,Bainer_Electricity_Demand_kBtu
6,6,Briggs Hall,AP.BRIGGS_Total_Count,Briggs_Electricity_Demand_kBtu
7,7,California Hall,AP.CALIFORNIA-HALL_Total_Count,California_Electricity_Demand_kBtu
8,8,Campus Data Center,AP.DATACENTER_Total_Count,CampusDataCenter_Electricity_Demand_kBtu
9,9,Chemistry,AP.CHEM_Total_Count,Chemistry_Electricity_Demand_kBtu


In [3]:
pd.set_option('display.max_rows', 500)
elec_tags = pd.read_csv('elec_tags.csv')
elec_tags

,Electricity_Demand_kBtu,start,end,Status
0,AcademicSurge_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
1,ARC_Electricity_Demand_kBtu,2017-09-01,2018-09-01,complete
2,ArtAnnex_Electricity_Demand_kBtu,2019-09-01,2020-09-01,complete
3,Asmundson_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
4,Bainer_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
5,California_Electricity_Demand_kBtu,2019-02-01,2020-02-01,complete
6,CampusDataCenter_Electricity_Demand_kBtu,2020-09-01,2021-09-01,complete
7,Chemistry_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
8,Chemistry.Annex_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
9,Cole.B_Electricity_Demand_kBtu,2020-09-01,2021-09-01,complete


In [15]:
incomplete = ['ARC_Electricity_Demand_kBtu',
             'Cruess_Electricity_Demand_kBtu',
             'GSM_Electricity_Demand_kBtu',
             'Physics.Roessler_Electricity_Demand_kBtu',
             'Pitzer_Center_Electricity_Demand_kBtu',
             'Tupper_Electricity_Demand_kBtu',
             'VMIF_Electricity_Demand_kBtu',
             'Wellman_Kerr_Electricity_Demand_kBtu']


In [20]:
for tag in incomplete:
    elec_tags.loc[elec_tags['Electricity_Demand_kBtu'] == tag, 'Status'] = 'incomplete'
    

In [21]:
elec_tags

,Electricity_Demand_kBtu,start,end,Status
0,AcademicSurge_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
1,ARC_Electricity_Demand_kBtu,2017-09-01,2018-09-01,incomplete
2,ArtAnnex_Electricity_Demand_kBtu,2019-09-01,2020-09-01,complete
3,Asmundson_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
4,Bainer_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
5,California_Electricity_Demand_kBtu,2019-02-01,2020-02-01,complete
6,CampusDataCenter_Electricity_Demand_kBtu,2020-09-01,2021-09-01,complete
7,Chemistry_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
8,Chemistry.Annex_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
9,Cole.B_Electricity_Demand_kBtu,2020-09-01,2021-09-01,complete


In [22]:
elec_tags.to_csv('elec_tags.csv', index = False)

In [23]:
elec_tags2 = pd.read_csv('elec_tags.csv')

In [24]:
elec_tags2

,Electricity_Demand_kBtu,start,end,Status
0,AcademicSurge_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
1,ARC_Electricity_Demand_kBtu,2017-09-01,2018-09-01,incomplete
2,ArtAnnex_Electricity_Demand_kBtu,2019-09-01,2020-09-01,complete
3,Asmundson_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
4,Bainer_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
5,California_Electricity_Demand_kBtu,2019-02-01,2020-02-01,complete
6,CampusDataCenter_Electricity_Demand_kBtu,2020-09-01,2021-09-01,complete
7,Chemistry_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
8,Chemistry.Annex_Electricity_Demand_kBtu,2018-09-01,2019-09-01,complete
9,Cole.B_Electricity_Demand_kBtu,2020-09-01,2021-09-01,complete
